<img style="float: right;" src="http://www2.le.ac.uk/liscb1.jpg">
Scipy is the first place to look for general-purpose scientific functionality.  The Scipy library is enormous and varied, so covering all the features is a course unto itself.  Having said that, one of the most commonly used features in scipy is it's fitting routines, which we will now explore.

## Curve fitting

Scipy comes with a number of fitting routines.  One that can be extremely useful is `curve_fit`, which fits a function to a given set of data using a least-squares minimization.  Here, we'll fit some (made up) data to the Michaelis-Menten equation:  

$$\large V = \frac{V_{max}[S]}{K_m+[S]}$$  

First, we define the Michaelis-Menten equation:

In [ ]:
def michaelis_menten(s, km, vmax):
    return (vmax*s) / (km + s)

Now we pick some substrate concentrations we (would) do our measurements at:

In [ ]:
import numpy as np

In [ ]:
substrate_concentrations = np.array([0.01, 0.05, 0.1, 0.5, 1, 2, 3])

For convenience when plotting, we'll create a numpy array of 1000 values between 0 and 3 for our x-axis:

In [ ]:
substrate_concentration_range = np.linspace(0, 3, 1000)

Ok, lets make sure this looks right:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
km = 0.1
vmax = 0.5

initial_velocities = michaelis_menten(s=substrate_concentrations, km=km, vmax=vmax)
mm_curve = michaelis_menten(substrate_concentration_range, km, vmax)


# Plot all the things!
plt.scatter(substrate_concentrations, initial_velocities)
plt.plot(substrate_concentration_range, mm_curve)

plt.ylabel('Velocity')
plt.xlabel('initial [S]')

# In M-M kinetics, the Km is the substrate concentration where you've reached half-max rate
plt.vlines(km, ymin=0, ymax=vmax/2, linestyle='dashed')
plt.hlines(vmax/2, xmin=0, xmax=km, linestyle='dashed')

plt.xlim(xmin=0, xmax=3)
plt.ylim(ymin=0, ymax=0.6)

Ok, so lets now simulate some noisy data (with Normally distributed noise):

In [ ]:
number_of_concentrations = len(substrate_concentrations)
noise = (np.random.normal(loc=1, scale=0.1, size=number_of_concentrations))
simulated_data = michaelis_menten(substrate_concentrations, km, vmax,) * noise

ok, so how does our simulation look?

In [ ]:
plt.plot(substrate_concentration_range, mm_curve, label='ground truth')
plt.scatter(substrate_concentrations, simulated_data, label='simulated data', color='green')
plt.legend(loc='lower right');

Now we can try to fit our original equation.  In order to do this, we need to provide a guess for the parameters that the algorithm can start from.  For something as simple as the M-M equation, even quite bad guesses will do.

In [ ]:
initial_guess = (100, simulated_data[-1])  # Km and Vmax.  Note the Km is a truly horrible guess, given our data

In addition, we can provide the fitting algorithm with bounds - regions of allowed values for the parameters.  In this case, it's not necissary, but it's always good to have a sanity check (in this case, both parameters must be positive.)

In [ ]:
lower_bounds = (0, 0)  # Km, Vmax
upper_bounds = (np.inf, np.inf)  # Km, Vmax

In [ ]:
from scipy.optimize import curve_fit

fitted, covariance = curve_fit(f=michaelis_menten,
                               xdata=substrate_concentrations,
                               ydata=simulated_data,
                               p0=initial_guess,
                               bounds=(lower_bounds, upper_bounds)
                              )
print('Km:', fitted[0])
print('Vmax:',fitted[1])

Remember, always look at your data as much as possible!

In [ ]:
calculated_curve = michaelis_menten(substrate_concentration_range, fitted[0], fitted[1])

plt.plot(substrate_concentration_range, mm_curve, label='ground truth')
plt.scatter(substrate_concentrations, simulated_data, label='simulated data', color='green')
plt.plot(substrate_concentration_range, calculated_curve, label='fitted')

plt.legend(loc='lower right');

### Exercise 1: Did the fitting actually work?
The easiest way to see if the fitting function returned sensible values is to randomize the starting point. If it keeps giving the same numbers from various starting positions, it's more likely you can trust the results.
1. Generate a random starting value for $K_m$ from 0.01 to 10, and for $V_{max}$ from 0.01 to 10.  *Hint: use `np.random.uniform()`
2. Fit using these starting values and print both the starting value and resulting parameters.
3. Repeat for 10 total fits.  How does it look?

### Exercise 2:  Compute the fitting error
Compute the error of the fit.  
*Hint: Read the docstring of curve_fit using `.?`, and look at what the function returns*

__Bonus:__
Remake the plot, but shade the error bounds.  
1. The maximum positive error is when you subtract the standard deviation from the Km and add it to the Vmax
2. Calculate the curve for the maximum and minimum errors as we did above
3. Ask google how to fill between curves in matplotlib
4. Try setting `alpha=0.2` in the plotting function.

## Uncertainty

In the fit above, we used the covariance matrix to determine the error bounds.  What's not so apparent from this is that this only works for some fitting routines, is only valid if the errors are normally distributed, and is then only valid if the data is homoscedastic (the measurement error is the same for every point - which isn't really true in this case!)  

So what are we to do?
1.  Work out the analytical error for every fit, assuming the characteristics of the error (hard - lots of algebra)
2.  Use resampling to empirically determine the distribution of parameter values (hard - lots of computation)

Except #2 isn't hard - we have computers to do the computation for us.  The type of resampling we'll be doing is call bootstrapping.  The mathematical justification for why this works is (way) beyond this course.  If you follow a few rules, you can approximate the actual underlying distribution on almost any parameter, and then extract what you want directly.  Here are the rules:
1. Generate a new set of data by sampling, with replacement, from your current data set.
2. The new data set must have the same number of observations as the original (some data will get repeated, some left out)
3. Fit your curve with the new dataset
4. Repeat a few hundred to tens of thousands of times until the distribution stabilizes

An example in code should make this clear:

In [ ]:
lower_bounds = (0, 0)  # Km, Vmax
upper_bounds = (np.inf, np.inf)  # Km, Vmax
initial_guess = (1, simulated_data[-1])  # Km and Vmax.

bootstrap_kms =[]
bootstrap_vmaxs = []

iterations = 100

for _ in range(iterations):
    # Generate new sample indices with replacement 
    indices = np.arange(len(substrate_concentrations)-1)
    resample_indices = np.random.choice(indices, size=len(indices), replace=True)
    resampled_substrate_concentrations = substrate_concentrations[resample_indices]
    resampled_simulated_data = simulated_data[resample_indices]
    
    # Fit the curve with the resampled dataset
    bootstrap_fitted, bootstrap_covariance = curve_fit(f=michaelis_menten,
                                                       xdata=resampled_substrate_concentrations,
                                                       ydata=resampled_simulated_data,
                                                       p0=initial_guess,
                                                       bounds=(lower_bounds, upper_bounds)
                                                      )
    bootstrap_kms.append(bootstrap_fitted[0])
    bootstrap_vmaxs.append(bootstrap_fitted[1])

Now look at the results

In [ ]:
plt.figure(figsize=(14, 4))

plt.subplot(121)
plt.hist(bootstrap_kms, bins=100)
# plt.xlim(0,1)
plt.title('Km')

plt.subplot(122)
plt.hist(bootstrap_vmaxs, bins=100)
# plt.xlim(0,2)
plt.title('Vmax');

len(bootstrap_kms)

### Exercise 3: Confidence interval
The 90% confidence interval is the range of values that contain the central 90% of the distribution.  Extract the 90%  CI for the data above.
1. Make your life easier now, change the number of iterations above to 10000 and start it running.
2. Make sorted listes of bootstrap_kms and bootstrap_vmaxs
3. If you have 10000 sorted data points, the central 9000 data points make up your 90% CI - extract the lowest and hightest values from these 9000
4. Use `plt.scatter` to plot all 10000 points.  What do you notice?
5. Make two plots comparing the fitting error to the 90% CI.  First, plot the $K_m$ CI using the originally fitted $V_{max}$, then plot the $V_{max}$ CI using the originally fitted $K_m$.

One thing to note here:  The confidence intervals *are not centered on the prediction*, this means that, given our data, we're more certain about one bound than the other.  Because we've used random noise, your intervals may appear centered, but if you run the notebook again, you may be able to see that they're not.

### Exercise 4: Experimental replicates
Most experiments will have replicate data points, i.e. the researcher measures the initial rate several times at each substrate concentration.
1. Create a new list of substrate concentrations called `substrate_concentrations_with_replicates`, using each concentration three times (order doesn't matter)
2. Create new simulated data called `simulated_data_with_replicates` from above, adding the same amount of random noise as for `simulated_data`
3. Fit the data and calculate the 90% CI with the replicates.  Make plots of the distributions.
What do you notice?

## Model choice

One common difficulty in analysing data is choosing the correct model.  The most tempting choice is the model that fits our data best - *this is extremely dangerous.*  Here's the equation for homotropic alosterism (substrate activation or inhibition of the enzyme):  

$$\large V = \frac{V_{max}\frac{[S]}{K_{D1}} + \beta V_{max}\frac{[S]^2}{\alpha K_{D1}K_{D12}}}{1 + \frac{[S]}{K_{D1}} + \frac{[S]}{K_{D2}} + \frac{[S]^2}{\alpha K_{D1} K_{D2}}}$$  

Now, **we know that our system doesn't have any substrate inhibition or activation** - we created the data afterall - but lets see how this does...

In [ ]:
def michaelis_menten_ha(s, kd1, kd2, alpha, beta, vmax):
    '''
    The Michaelis-Menton function including homotropic allosterism. 
    '''
    numerator = vmax * (s/kd1) + (beta * vmax * (s**2)/(alpha * kd1 *kd2))
    denominator = 1 + (s/kd1) + (s/kd2) + (s**2)/(alpha * kd1 * kd2)
    
    return numerator/denominator

If we set $\alpha$ and $\beta$ to be 1, and set $K_{D1}$ and $K_{D2}$ to the same value, the equation above reduces to the standard Michaelis-Menton equation, so we'll set our starting parameters there.

In [ ]:
initial_guess = (fitted[0], fitted[0], 1, 1, fitted[1])  # KD1, KD2, alpha, beta and Vmax
lower_bounds_ha = (0, 0, 0, 0, 0)
upper_bounds_ha = (np.inf, np.inf, np.inf, np.inf, np.inf)

fitted_ha, covariance_ha = curve_fit(f=michaelis_menten_ha,
                               xdata=substrate_concentrations,
                               ydata=simulated_data,
                               p0=initial_guess,
                               bounds=(lower_bounds_ha, upper_bounds_ha)
                              )

print('Original Michaelis-Menton')
print('Km:', fitted[0])
print('Vmax:',fitted[1])

print()

print('Michaelis-Menton with aditional parameters')
print('KD1', fitted_ha[0])
print('KD2:',fitted_ha[1])
print('alpha:', fitted_ha[2])
print('beta:',fitted_ha[3])
print('Vmax:', fitted_ha[4])

Ok, so now which one fits our data better?

In [ ]:
calculated_curve_ha = michaelis_menten_ha(substrate_concentration_range,
                                          *fitted_ha)  # *fitted_ha => fitted_ha[0], fitted_ha[1]

plt.figure(figsize=(14,4))

plt.subplot(121)
plt.scatter(substrate_concentrations, simulated_data, label='simulated data', color='green')
plt.plot(substrate_concentration_range, calculated_curve, label='Fitted Michaelis-Menton', color='red')
plt.legend(loc='lower right')

plt.subplot(122)
plt.scatter(substrate_concentrations, simulated_data, label='simulated data', color='green')
plt.plot(substrate_concentration_range, calculated_curve_ha, label='Fitted Michaelis-Menton-HA', color='purple')
plt.legend(loc='lower right');

Which one do you think looks better?

So what does this mean?  Because the  `michaelis_menten_ha` function is the `michaelis_menten` function with three extra parameters, it will **always** fit noisy data better.  Because we used the `michaelis_menten` function to generate the data, we know that's the one we should actually be using, though.  If you're faced with a real-world problem, and the function with more parameters will always fit noisy data better, how can you tell if you really should use the function with more parameters or not?

## Model selection by cross validation

How do you know which model to choose?  One of the best ways is cross-validation.  The key to undersanding why this is so powerful is that cross-validation essentially tests how much of the random noise you're modelling.  The function that models the least noise and most signal is almost always the correct one. 

Cross-validation leaves a bit of data out, then fits (trains) the curve without that data, then tests how well the left out data is predicted by the fit.

We'll start with leave-one-out cross validation:

In [ ]:
initial_guess = (100, simulated_data[-1])  # Km and Vmax.  Note the Km is a truly horrible guess, given our data
lower_bounds = (0, 0)  # Km, Vmax
upper_bounds = (np.inf, np.inf)  # Km, Vmax
predictions = []

# Step through each substrate concentration
for i in range(len(substrate_concentrations)):
    
    # Build a mask excluding the test measurement
    mask = np.ones(len(substrate_concentrations)).astype('bool')
    mask[i] = False
    
    # Remove the test measurement from our training data
    training_substrate_concentrations = substrate_concentrations[mask]
    training_simulated_data = simulated_data[mask]
    
    # Fit the curve
    bootstrap_fitted, bootstrap_covariance = curve_fit(f=michaelis_menten,
                                                       xdata=training_substrate_concentrations,
                                                       ydata=training_simulated_data,
                                                       p0=initial_guess,
                                                       bounds=(lower_bounds, upper_bounds)
                                                      )
    predictions.append(michaelis_menten(substrate_concentrations[i], *fitted))

abs_errors = abs(predictions - simulated_data)
xval_error = sum(abs_errors)
print("Cross validation error for 'michaelis_menten':",xval_error)

### Exercise 5: Model selection by cross validation
Run the cross validation on `michaelis_menten_ha`.  Which function gives lower cross-validation error?  

*Hint: If you get `Optimal parameters not found` error, add the parameter `max_nfev=10000` to the `curve_fit` call.*


So which model do we choose?

In [ ]:
xval_ratio = xval_error / xval_ha_error

if xval_ratio < 0.95:
    print('Michaelis-Menton is the prefered model.')
elif xval_ratio > 1.05:
    print('Michaelis-Menton with Homotropic Alostery is the prefered model.')
else:
    print("The cross validation scores are too close, we can't descriminiate between the models")

As always, there are lots of choices for how exactly we do cross-validation, but here are the two most import issues:

1. If you only have a few more observations than parameters, do leave-one-out validation.
2. If you have correlated data (i.e. several blocks of data,) try to keep the correlated data either in the test or training sets  

In general, if you have many more data points than parameters, you can do k-fold cross validation.  For example, with 5-fold cross-validation, you would split your data into 5 chunks, using 4 of the chunks for training and one for error determination.  As you have 5 chunks, you do it 5 times (using a different chunk each time,) and sum up the errors.  The model with the lowest error is the one over-fitting the least.